In [657]:
import ipywidgets as widgets
from IPython.display import display
from altair import value

In [658]:
# Constants
BACKGROUND_IMAGE_PATH = "nhl_rink.png"
ICON_PATH = "icon.png"
# NHL RINK
BACKGROUND_WIDTH = 1100
BACKGROUND_HEIGHT = 467

# NHL API
X_COORD_MIN = -100
X_COORD_MAX = 100
Y_COORD_MIN = -42.5
Y_COORD_MAX = 42.5

ICON_SIZE = 25
# Needed to make sure icon doesn't go out of bounds. Need to set icon back 1* its size to be within bounds + 0.5* its size for it to be centered
ICON_LIMIT_RATIO = 1.5
# The Layout system has a weird +- 3px in the y direction for invisible borders. Needs to be removed from start position in y
X_COORD_TO_PX_RATIO = (BACKGROUND_WIDTH - ICON_SIZE * ICON_LIMIT_RATIO) / (X_COORD_MAX - X_COORD_MIN)
X_PX_TO_COORD_RATIO = 1.0 / X_COORD_TO_PX_RATIO
Y_COORD_TO_PX_RATIO = (BACKGROUND_HEIGHT - ICON_SIZE * ICON_LIMIT_RATIO) / (Y_COORD_MAX - Y_COORD_MIN)
Y_PX_TO_COORD_RATIO = 1.0 / Y_COORD_TO_PX_RATIO
EVENT_TYPES = {"icon": ICON_PATH, "goal": "goal.png", "missed_shot": "missed_shot.png"}

In [659]:
background_image = widgets.Image(
    value=open(BACKGROUND_IMAGE_PATH, "rb").read(),
    format='png',
    width=BACKGROUND_WIDTH,
    height=BACKGROUND_HEIGHT
)

In [660]:
# Create the overlay icon
overlay_icon = widgets.Image(
    value=open(ICON_PATH, "rb").read(),
    format='png',
    width=ICON_SIZE,
    height=ICON_SIZE
)

In [661]:
# Create the sliders and debug info
x_slider = widgets.FloatSlider(
    value=0,
    min=X_COORD_MIN,
    max=X_COORD_MAX,
    step=1,
    description='X Position'
)
y_slider = widgets.FloatSlider(
    value=0,
    min=Y_COORD_MIN,
    max=Y_COORD_MAX,
    step=1,
    description='Y Position'
)

debug_label = widgets.Label(value = 'debug')
event_type_dropdown = widgets.Dropdown(
    options=EVENT_TYPES.keys(),
    value='icon',
    description='Event Type:'
)

In [662]:
# Functions to update the position and the event type
def update_position(change):
    # There is a discrepancy between the possible values.
    # possible value in the data for x is -100 to 100.
    # possible value in the ui for x is 0px to 1100px.
    # possible value in the data for x is -42.5 to 42.5 .
    # possible value in the ui for y is 0px to 467px.
    new_x_offset = x_slider.value * X_COORD_TO_PX_RATIO
    new_y_offset = y_slider.value * Y_COORD_TO_PX_RATIO
    overlay_icon.layout.left = f'{ x_initial + new_x_offset}px'
    
    overlay_icon.layout.top = f'{y_initial + new_y_offset}px'
    debug_label.value = f'new coords for layout: {overlay_icon.layout.left}, {overlay_icon.layout.top}'
    debug_label.value += f'\n size of overlay: {overlay_icon.layout.width}, {overlay_icon.layout.height}'

def update_event_type(change):
    new_icon_path = EVENT_TYPES[event_type_dropdown.value]
    overlay_icon.value = open(new_icon_path, "rb").read()
    debug_label.value = f'New Event: {event_type_dropdown.value}'

x_slider.observe(update_position, names='value')
y_slider.observe(update_position, names='value')
event_type_dropdown.observe(update_event_type, names='value')

In [663]:
# Calculate the center of the rink
background_x_center = int(background_image.width) / 2
background_y_center = int(background_image.height) / 2

# Keep in mind that the icon is displayed to the right edge of the background with left = 0
x_initial = 0 - (background_x_center)
y_initial = background_y_center - ICON_SIZE

In [664]:
overlay_icon.layout = widgets.Layout(
    position='absolute',
    width = f'{ICON_SIZE}px',
    height = f'{ICON_SIZE}px',
    top=f'{y_initial}px',  # 50% of the height minus half the icon height
    left=f'{x_initial}px'  # 50% of the width minus half the icon width
)

In [665]:
container = widgets.Box(
    children=[background_image, overlay_icon],
    layout=widgets.Layout(
        width=f'{BACKGROUND_WIDTH}px',
        height=f'{BACKGROUND_HEIGHT}px',
        position='absolute'
    )
)

In [666]:
# Display the main container
display(container, x_slider, y_slider, debug_label, event_type_dropdown)

Box(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04L\x00\x00\x01\xd3\x08\x03\x00\x00\x…

FloatSlider(value=0.0, description='X Position', min=-100.0, step=1.0)

FloatSlider(value=0.0, description='Y Position', max=42.5, min=-42.5, step=1.0)

Label(value='debug')

Dropdown(description='Event Type:', options=('icon', 'goal', 'missed_shot'), value='icon')